In [1]:
import os
import sys
import pandas as pd
import requests
import logging
from datetime import datetime, timedelta
import random
import time
import psycopg2
from io import StringIO
from dotenv import load_dotenv

# Add project root to sys.path
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Verify sys.path
print("Current sys.path:", sys.path)

from src.data_processing.nst_scraper import nst_on_ice_scraper, nst_team_on_ice_scraper
from src.db.nst_db_utils import insert_goalie_stats_df, scrape_goalie_stats_range, scrape_team_stats_range


pd.set_option('display.max_columns', None)


Current sys.path: ['C:\\Users\\H-100\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\H-100\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\H-100\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\H-100\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv', '', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv\\lib\\site-packages', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv\\lib\\site-packages\\win32', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper\\.venv\\lib\\site-packages\\Pythonwin', 'c:\\Users\\H-100\\Desktop\\python\\accurateshothelper']


In [2]:
# nst_team_on_ice_scraper(startdate='', enddate='2024-10-12',last_n=30)
# df = nst_on_ice_scraper(startdate='', enddate='2024-10-09', last_n=30, rate='y', lines='single') 

In [3]:
# goalie_stats_df = nst_on_ice_scraper(
#             startdate='2024-10-12',
#             enddate='2024-10-12',
#             pos='G',
#             rate='n',
#             stdoi='g',
#             lines='single'
#         )


In [4]:
# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [5]:
# Load environment variables from .env file
load_dotenv()

db_prefix = 'NST_DB_'

In [6]:
# NHL_SEASONS = {
#     20242025: {
#         'start': '2024-10-04',
#         'regular_end': '2025-04-18',  # Estimated
#         'playoff_end': '2025-06-30'  # Estimated
#     },
#     20232024: {
#         'start': '2023-10-10',
#         'regular_end': '2024-04-18',
#         'playoff_end': '2024-06-24'
#     },
#     20222023: {
#         'start': '2022-10-07',
#         'regular_end': '2023-04-14',
#         'playoff_end': '2023-06-13'
#     },
#     20212022: {
#         'start': '2021-10-12',
#         'regular_end': '2022-04-29',
#         'playoff_end': '2022-06-26'
#     },
#     20202021: {
#         'start': '2021-01-13',
#         'regular_end': '2021-05-19',
#         'playoff_end': '2021-07-07'  # Covid-shortened season
#     }

In [7]:
# scrape_goalie_stats_range(
#         start_date='2024-10-04',
#         end_date='2024-10-06',
#         db_prefix=db_prefix  # Make sure this matches your environment variables
#     )

In [8]:
# scrape_goalie_stats_range(
#         start_date='2023-03-01',
#         end_date='2023-04-14',
#         db_prefix=db_prefix,  # Make sure this matches your environment variables
#         delay_min=90,
#         delay_max=120,
#         situation='pk'
#     )

In [9]:
# nst_team_on_ice_scraper(
#                 startdate='2025-02-24',
#                 enddate='2025-02-24',
#                 sit='5v5',
#                 stype=2
#             )


In [18]:
scrape_team_stats_range(
    start_date='2024-10-04',
    end_date='2024-12-31',
    db_prefix=db_prefix,
    delay_min=90,
    delay_max=120,
    situation='5v5'
)


INFO:src.db.nst_db_utils:Scraping team data for date: 2024-10-04
INFO:src.db.nst_db_utils:Team Stats DataFrame shape: (2, 73)
INFO:src.db.nst_db_utils:Team Stats DataFrame columns: ['team', 'gp', 'toi', 'w', 'l', 'otl', 'row', 'points', 'point_%', 'cf', 'ca', 'cf%', 'ff', 'fa', 'ff%', 'sf', 'sa', 'sf%', 'gf', 'ga', 'gf%', 'xgf', 'xga', 'xgf%', 'scf', 'sca', 'scf%', 'scsf', 'scsa', 'scsf%', 'scgf', 'scga', 'scgf%', 'scsh%', 'scsv%', 'hdcf', 'hdca', 'hdcf%', 'hdsf', 'hdsa', 'hdsf%', 'hdgf', 'hdga', 'hdgf%', 'hdsh%', 'hdsv%', 'mdcf', 'mdca', 'mdcf%', 'mdsf', 'mdsa', 'mdsf%', 'mdgf', 'mdga', 'mdgf%', 'mdsh%', 'mdsv%', 'ldcf', 'ldca', 'ldcf%', 'ldsf', 'ldsa', 'ldsf%', 'ldgf', 'ldga', 'ldgf%', 'ldsh%', 'ldsv%', 'sh%', 'sv%', 'pdo', 'date', 'season']
INFO:src.db.base_utils:Database connection established.
INFO:src.db.nst_db_utils:Cleaned columns: ['team', 'gp', 'toi', 'w', 'l', 'otl', 'row', 'points', 'point_pct', 'cf', 'ca', 'cf_pct', 'ff', 'fa', 'ff_pct', 'sf', 'sa', 'sf_pct', 'gf', 'ga', '